# Exporting CSV to PostgreSQL with Data Transformations
This notebook demonstrates how to load data from a CSV file, apply transformations based on your observations, and export it to a PostgreSQL database.

## Key Observations and Updates:
1. **NaN Values**: Keep `NaN` for categorical columns and replace missing values with `None` for numerical columns.
2. **Binary Transformation**: 
   - `Alert`: Binary (0 or 1).
   - `BearBreak`: Transform "Bearish Breakout" to `1` and empty values to `0`.
3. **Date/Time Format**: Convert `Date/Time` to a proper `datetime` format.
4. **Ensure Data Types**: Ensure consistency in data types across columns (e.g., `Alert` as float, `BearBreak` as binary, etc.).


In [ ]:
# Import necessary libraries
import pandas as pd
import psycopg2

# Function to explore dataset
# You can uncomment this to explore the dataset before applying transformations

# def explore_dataset(df):
#     """
#     Function to show unique values, data types, and NaN counts for each column in the DataFrame.
#     """
#     for column in df.columns:
#         unique_values = df[column].unique()
#         num_unique = len(unique_values)
#         num_missing = df[column].isna().sum()
#         dtype = df[column].dtype

#         # Display the column's name, data type, and number of unique values
#         print(f"\nColumn: {column}")
#         print(f"Data Type: {dtype}")
#         print(f"Number of Unique Values: {num_unique}")
#         print(f"Number of Missing Values: {num_missing}")
#         print(f"Unique Values: {unique_values[:10]}")  # Display the first 10 unique values only for readability

#         # If there are more than 10 unique values, just print a note
#         if num_unique > 10:
#             print(f"... and {num_unique - 10} more unique values")

# Load DataFrame from CSV and specify column types
def load_from_csv():
    """
    Load CSV and specify column types for certain fields.
    """
    return pd.read_csv(
        'merged_tickers.csv',
        dtype={
            'Alert': 'float', 
            'BearBreak': 'str', 
            'Buy_ATR': 'str',
            'Buy_BB': 'str', 
            'Buy_EW': 'str', 
            'Buy_LR': 'str', 
            'Buy_ZZ': 'str',
            'Pattern': 'str'
        },
        low_memory=False
    )

# Prepare DataFrame for insertion by converting Date/Time, replacing NaN, and ensuring correct data types
def prepare_data_for_insert(df):
    """
    Prepare data by applying transformations:
    1. Convert Date/Time to datetime.
    2. Replace NaN with None for numerical values and keep NaN for categorical columns.
    3. Binary conversion for 'Alert' and 'BearBreak'.
    4. Ensure proper data types for all columns.
    """
    # Convert 'Date/Time' to datetime format
    df['Date/Time'] = pd.to_datetime(df['Date/Time'], errors='coerce')

    # Replace NaN with None for numerical columns while keeping NaN for categorical columns
    df = df.where(pd.notna(df), None)
    
    # Handle binary conversion
    df['Alert'] = df['Alert'].astype(float, errors='ignore')  # Ensure 'Alert' is binary (0 or 1)
    
    # Convert 'BearBreak' to binary: 1 for 'Bearish Breakout' and 0 for others
    df['BearBreak'] = df['BearBreak'].apply(lambda x: 1 if x == 'Bearish Breakout' else 0)

    # Ensure the correct types for other columns
    df['Buy_ATR'] = df['Buy_ATR'].astype(str, errors='ignore')
    df['Buy_BB'] = df['Buy_BB'].astype(str, errors='ignore')
    df['Buy_EW'] = df['Buy_EW'].astype(str, errors='ignore')
    df['Buy_LR'] = df['Buy_LR'].astype(str, errors='ignore')
    df['Buy_ZZ'] = df['Buy_ZZ'].astype(str, errors='ignore')
    df['Pattern'] = df['Pattern'].astype(str, errors='ignore')

    return df

# Function to insert DataFrame into PostgreSQL
def insert_to_postgres(df):
    """
    Connect to PostgreSQL and insert the DataFrame.
    """
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            host="localhost",  # your host
            database="postgres",  # your database
            user="postgres",  # your username
            password="your_password",  # replace with your PostgreSQL password
            port="5432"  # default port for PostgreSQL
        )
        
        cur = conn.cursor()

        # Create table in PostgreSQL if it doesn't exist
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS joined_tickers (
            ticker TEXT,
            date_time TIMESTAMP,
            buy TEXT,
            alert NUMERIC,
            bearbreak NUMERIC,  -- Binary (1 for Bearish Breakout, 0 otherwise)
            buy_atr TEXT,
            buy_bb TEXT,
            buy_ew TEXT,
            buy_lr TEXT,
            buy_zz TEXT,
            close NUMERIC,
            dnbars NUMERIC,
            downtrend_length NUMERIC,
            fromhigh NUMERIC,
            fromlow NUMERIC,
            lr_lc NUMERIC,
            lr_mc NUMERIC,
            lr_uc NUMERIC,
            mega_sellby_larger_wave NUMERIC,
            numberoftickers NUMERIC,
            pattern TEXT,
            percentilerank NUMERIC,
            rank NUMERIC,
            s1 NUMERIC,
            sctr NUMERIC,
            sl NUMERIC,
            upbars NUMERIC,
            uptrend_length NUMERIC,
            hh NUMERIC,
            ll NUMERIC,
            myshort NUMERIC
        );
        '''
        cur.execute(create_table_query)
        conn.commit()

        # Insert data into PostgreSQL
        insert_query = '''
        INSERT INTO joined_tickers (ticker, date_time, buy, alert, bearbreak, buy_atr, buy_bb, buy_ew, buy_lr, buy_zz, close, 
            dnbars, downtrend_length, fromhigh, fromlow, lr_lc, lr_mc, lr_uc, mega_sellby_larger_wave, numberoftickers, 
            pattern, percentilerank, rank, s1, sctr, sl, upbars, uptrend_length, hh, ll, myshort)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''

        # Iterate over DataFrame and insert each row
        for _, row in df.iterrows():
            values = (
                row['Ticker'], 
                row['Date/Time'], 
                row['Buy'], 
                row.get('Alert', None),
                row.get('BearBreak', None),
                row.get('Buy_ATR', None),
                row.get('Buy_BB', None),
                row.get('Buy_EW', None),
                row.get('Buy_LR', None),
                row.get('Buy_ZZ', None),
                row.get('Close', None),
                row.get('DnBars', None),
                row.get('Downtrend Length', None),
                row.get('FromHigh', None),
                row.get('FromLow', None),
                row.get('LR_LC', None),
                row.get('LR_MC', None),
                row.get('LR_UC', None),
                row.get('Mega Sellby Larger Wave', None),
                row.get('NumberOfTickers', None),
                row.get('Pattern', None),
                row.get('PercentileRank', None),
                row.get('Rank', None),
                row.get('S1', None),
                row.get('SCTR', None),
                row.get('SL', None),
                row.get('UpBars', None),
                row.get('Uptrend Length', None),
                row.get('hh', None),
                row.get('ll', None),
                row.get('myShort', None)
            )

            cur.execute(insert_query, values)

        conn.commit()

        # Verify data insertion
        cur.execute("SELECT * FROM joined_tickers LIMIT 10")
        rows = cur.fetchall()
        for row in rows:
            print(row)

        # Close the cursor and connection
        cur.close()
        conn.close()

    except Exception as error:
        print(f"Error: {error}")

# Load and prepare the data
df = load_from_csv()

# Uncomment the next line if you want to explore the dataset before transformation
# explore_dataset(df)

df = prepare_data_for_insert(df)

# Insert the DataFrame into PostgreSQL
insert_to_postgres(df)


## Next Steps:
1. **Run the code** in this notebook to load, transform, and export the data into PostgreSQL.
2. **Verify the Data**: After exporting, query the PostgreSQL database to verify that the transformations were correctly applied and the data is properly inserted.
3. **Explore Dataset**: If you want to explore the dataset before applying transformations, uncomment the `explore_dataset(df)` line.
